# Escuela de Verano 2020

# Extracción de medicamentos a partir de notas clínicas

### Objectivo:
A partir de un conjunto de textos especializados (notas clínicas) extraer los tratamientos clínicos administrados a pacientes con enfermedades cardiovasculares y respiratorias.

#### Descripción de la base de datos:
Se extrajeron un total de 70 notas clínicas de pacientes con enfermedades cardiovasculares del sitio [MTSAMPLES](https://www.mtsamples.com).

Las notas corresponden a consultas médicas, reportes de procedimientos y cartas médicas. Cada una de las notas describe los medicamentos administrados a los pacientes.

## Carga de datos

In [1]:
import pandas as pd

url_notas = '../notas/data_set.csv' #ubicación de las notas clínicas
df = pd.read_csv(url_notas)
print("Tamaño del dataset :", df.shape)
print(df.head())

Tamaño del dataset : (70, 2)
   subject_id                                               text
0         101  A lady was admitted to the hospital with chest...
1         102  Patient with palpitations and recent worsening...
2         103  The patient is a very pleasant 62-year-old Afr...
3         104  Patient with significant angina with moderate ...
4         105  This 57-year-old black female was seen in my o...


In [2]:
#Revisamos algunas notas
for i in range(2):
    print("----------------------------------------------")
    print(f"Nota: {i+1}")
    print(df.loc[i, 'text'])

----------------------------------------------
Nota: 1
A lady was admitted to the hospital with chest pain and respiratory insufficiency. She has chronic lung disease with bronchospastic angina. (Medical Transcription Sample Report) Ms. ABCD is a 69-year-old lady, who was admitted to the hospital with chest pain and respiratory insufficiency. She has chronic lung disease with bronchospastic angina. We discovered new T-wave abnormalities on her EKG. There was of course a four-vessel bypass surgery in 2001. We did a coronary angiogram. This demonstrated patent vein grafts and patent internal mammary vessel and so there was no obvious new disease. She may continue in the future to have angina and she will have nitroglycerin available for that if needed. Her blood pressure has been elevated and so instead of metoprolol, we have started her on Coreg 6.25 mg, captopril and heparin. This should be increased up to 25 mg b.i.d. as preferred antihypertensive in this lady's case. She also is on P

In [3]:
"""Pre-procesamiento de las notas clínicas"""
import re

def clean(sentencia):
    sentencia_minus = sentencia.lower() #se convierten el texto a minúsculas
    sentence_corregida = re.sub('and/or', 'or', sentencia_minus) #increase captopril and/or lopressor
    sentence_corregida = re.sub('(?<=[a-zA-Z])/(?=[a-zA-Z])', ' or ', sentence_corregida) #captopril/lopressor
    #sentence_corregida = re.sub(r'\W', ' ', sentence_corregida)
    sentencia_lista = sentence_corregida.replace("..", ".") #se reemplaza puntos suspensivos por un punto
    punctuation_spacer = str.maketrans({key: f"{key} " for key in ".,"}) #se inserta un espacio después de punto o coma
    sentencia_lista = sentencia_lista.translate(punctuation_spacer)
    sentencia_lista = ' '.join(sentencia_lista.split()) #se convierte múltiples espacios a un espacio
    return sentencia_lista

## Analizando las notas clínicas con BioC y NegBio

#### ¿Qué es BioC?
- Es una biblioteca para convertir datos clínicos a un formato estándar que es usado para tareas de serialización y deserialización de datos en formato XML. Para más detalles visitar su [página oficial](http://bioc.sourceforge.net/).

Particularmente, utilizaremos el objeto `BioCCollection`, el cual toma una colección de datos y los serializa en formato cadena. La colección puede ser un corpus completo o parcial. Además se utilizará la función `text2bioc()` la cual se encarga de transformar el texto hacia formato XML de BioC.


In [4]:
import bioc

collection = bioc.BioCCollection()
print(f"Atributos: \n\n{collection.__dict__}\n")
print(f"Métodos: \n\n{dir(collection)}\n")
print(f"Número de documentos en la colección: {collection.documents}")

Atributos: 

{'encoding': 'utf-8', 'version': '1.0', 'standalone': True, 'source': '', 'date': '2020-09-02', 'key': '', 'infons': {}, 'documents': []}

Métodos: 

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_document', 'clear_infons', 'date', 'documents', 'encoding', 'infons', 'key', 'source', 'standalone', 'version']

Número de documentos en la colección: []


#### ¿Qué es NegBio?
- Es una biblioteca diseñada para distinguir hallazgos negativos o inciertos en textos clínicos, particularmente en informes de radiología. NegBio trabaja particularmente con patrones sobre dependencias universales. Para más detalles visitar el sitio de [GitHub](https://github.com/ncbi-nlp/NegBio) o su [documentación oficial](https://negbio.readthedocs.io/en/latest/index.html).

Usaremos la función `NegBioSSplitter()` la cual se encarga de separar los textos en sentencias para su procesamiento.

In [5]:
from negbio.pipeline.ssplit import NegBioSSplitter
from negbio.pipeline import text2bioc
import StanfordDependencies

def colleccion_bioc_notas(df):
    collection = bioc.BioCCollection() #toma una colección de datos y los serializa a cadena
    split = NegBioSSplitter()#se instancia la función para separar los textos
    for i, report in enumerate(df["text"]):
        sentencia = text2bioc.text2document(str(i), clean(report)) #transforma la sentencia en formato XML
        sentencia = split.split_doc(sentencia)#separa los textos en sentencias
        collection.add_document(sentencia) #se añade la sentencia a la colección
    return collection

In [6]:
"""Transformación de las notas clínicas en formato Bio, pasando por la función de pre-procesamiento"""
collection = colleccion_bioc_notas(df)

In [19]:
len(collection.documents)

70

## Importar dependencias de NegBio y de la ontología clínica



In [9]:
from pathlib2 import Path
from negbio.neg.neg_detector import Detector
from negbio.main_mm import pipeline
from negbio.cli_utils import parse_args, get_absolute_path
from negbio.pipeline import negdetect, text2bioc, dner_mm
from negbio.negbio_dner_matamap import read_cuis
from negbio.pipeline.parse import NegBioParser
from negbio.pipeline.ssplit import NegBioSSplitter
from negbio.pipeline.ptb2ud import NegBioPtb2DepConverter, Lemmatizer
import pymetamap

#NegBio emplea el parser BLIPP, el cual es un analizador estadístico de lenguaje natural.
PARSING_MODEL_DIR = "~/.local/share/bllipparser/GENIA+PubMed" #corpus biomédicos anotados

#Se importan los archivos con los patrones de negación e incerdumbre
POST_NEG_PATH = "../NegBio/negbio/patterns/uncertainty_patterns.txt"
NEG_PATH = "../NegBio/negbio/patterns/neg_patterns.txt"

In [10]:
!cat $NEG_PATH


{} >{dependency:/neg/} {}
{} >{} {lemma:/no/}
{} >{dependency:/case/} {lemma:/without/}

# rather than XXX
{} <{dependency:/conj:negcc/} {}
{} <{dependency:/nmod:without/} {}
{} <{dependency:/conj:versus/} {}
{} <{dependency:/nmod:without|nmod:of/} {lemma:/clear|clearing/}=key
{} <{dependency:/nmod:out/} {lemma:/rule/}=key
{} <{dependency:/nmod:of/} {lemma:/history|free|disappearance|resolution|drainage|resolution|removal/}
{} <{dependency:/nmod:for/} {lemma:/negative/}
{} <{} {lemma:/resolve|resolving|exclude/}=key
{} <{dependency:/advmod|dep|conj:or/} {lemma:/no/}

# XXX has resolved
{} <{dependency:/nsubj/} ({lemma:/resolve/}=key >{dependency:/aux/} {})

# there is no XXX
{} <{dependency:/nsubj/} ({lemma:/be/} >{} {lemma:/no/})

# without evidence|finding of|for XXX
{} <{dependency:/nmod:of|nmod:for/} ({lemma:/evidence|finding/} <{dependency:/nmod:without/} {})

# no evidence of|for XXX
{} <{dependency:/nmod:of|nmod:for/} ({lemma:/evidence/} >{dependency:/neg/} {})

# without evide

#### Instanciamos UMLS e indicamos los medicamentos a extraer

Para que la ontología funcione correctamente, debe tomar en cuenta que:
- La ontología esté instalada localmente [Descarga](https://metamap.nlm.nih.gov/MainDownload.shtml)
- Los servidores `skrmedpostctl` y `wsdserverctl` deben estar activos [Instalación](https://metamap.nlm.nih.gov/Installation.shtml)

In [11]:
"Instanciamos la ontología"
ontologia_clinica = pymetamap.MetaMap.get_instance("/home/blanca/Documents/METAMAP_Lite/public_mm/bin/metamap20")
"Indicamos los CUIS de los medicamentos a extraer"
url_cuis= '../notas/cuis.txt'
cuis = read_cuis(url_cuis)#se lee los CUIS de los medicamentos que serán extraídos

In [12]:
"Se instancia el lematizador de NegBio"
lemmatizer = Lemmatizer() #Devuelve el lema de cada palabra.

Vamos a instanciar la función de `NegBioPtb2DepConverter()` de NegBio.
Convierte de un análisis de árbol hacia dependencias universales usando la herramienta de [Stanford converter](https://github.com/dmcc/PyStanfordDependencies)

- El análisis de árbol usado es [`Penn Treebank`](https://catalog.ldc.upenn.edu/docs/LDC95T7/cl93.html): corpus anotado que incluye etiquetas adicionales al etiquetado del POS.
- Las [`dependencias universales`](https://universaldependencies.org/): proporcionan un marco de trabajo para anotar la gramática en diferentes idiomas.

In [13]:
ptb2dep = NegBioPtb2DepConverter(lemmatizer, universal=True)
ssplitter = NegBioSSplitter(newline=True) ##se instancia la función para separar los textos
parser = NegBioParser(model_dir=PARSING_MODEL_DIR) #"~/.local/share/bllipparser/GENIA+PubMed"

Se instancia la función `Detector()` de NegBiola cual nos ayudará a determinar si existe alguna negación o incertidumbre en las notas, para lograr esto se pasan los patrones identificados previamente

In [14]:
negdetector = Detector(NEG_PATH, POST_NEG_PATH)

## ¡Unimos todo!
Usando la función `pipeline()` tomará como entrada todos los objetos instanciados previamente e identificaremos los medicamentos en las notas clínicas

In [15]:
collection = pipeline(collection, ontologia_clinica, ssplitter,
                      parser, ptb2dep, negdetector, cuis, None)

In [16]:
collection.documents[0]

BioCDocument[id=0,infons=[],passages=[BioCPassage[offset=0,text='a lady was admitt ... lasix and aspirin',infons=[],sentences=[],annotations=[BioCAnnotation[id=0,text='metoprolol',infons=[CUI=C0025859,semtype=orch,phsu,term=metoprolol,annotator=MetaMap],locations=[BioCLocation[offset=761,length=10]],],BioCAnnotation[id=1,text='captopril',infons=[CUI=C0006938,semtype=aapp,phsu,term=captopril,annotator=MetaMap],locations=[BioCLocation[offset=812,length=9]],],BioCAnnotation[id=2,text='heparin',infons=[CUI=C0019134,semtype=bacs,orch,phsu,term=heparin,annotator=MetaMap],locations=[BioCLocation[offset=826,length=7]],],BioCAnnotation[id=3,text='lasix',infons=[CUI=C0699992,semtype=orch,phsu,term=Lasix,annotator=MetaMap],locations=[BioCLocation[offset=980,length=5]],],BioCAnnotation[id=4,text='aspirin',infons=[CUI=C0004057,semtype=orch,phsu,term=aspirin,annotator=MetaMap],locations=[BioCLocation[offset=990,length=7]],],BioCAnnotation[id=5,text='clopidogrel',infons=[CUI=C0070166,semtype=orch,phs

## Visualizamos los resultados en un dataframe
Con el objetivo de obtener los resultados de forma más clara, construidos un matriz que resume los medicamentos administrados para cada paciente. Cada fila indica un paciente y cada columna un medicamento, la intersección entre columnas y filas describe `1` si el medicamento fue administrado y `0` si el medicamento no fue administrado.

In [17]:
def checkKey(dict, key):
    """ funcion para revisar la existencia de la etiqueta 'annotation''"""
    i=1
    if key in dict.keys():
        print("Present, ", end =" ")
        print("value =", dict[key])
        i=1
    else:
        i=0
    return i

negbio_pred = pd.DataFrame()
present = ''
term = ''
for doc in collection.documents:
    dictionary = {}
    for i in doc.passages:
        annotation = i.annotations
        for j in annotation:
            infons = j.infons
            print(infons)
            flag_neg = checkKey(infons, 'negation')
            if flag_neg == 0:
                for key, val in infons.items():
                    if key == 'term':
                        term = val
                        dictionary[term] = 'Positive'
            if flag_neg == 1: 
                for key, val in infons.items():
                    if key == 'term':
                        term = val
                        dictionary[term] = 'Negative'
    print("================")
    negbio_pred = negbio_pred.append(dictionary, ignore_index=True)

negbio_pred = negbio_pred.replace("Positive", 1).replace("Negative", 0).fillna(0)
negbio_pred.to_csv("prueba.csv")

{'CUI': 'C0025859', 'semtype': 'orch,phsu', 'term': 'metoprolol', 'annotator': 'MetaMap'}
{'CUI': 'C0006938', 'semtype': 'aapp,phsu', 'term': 'captopril', 'annotator': 'MetaMap'}
{'CUI': 'C0019134', 'semtype': 'bacs,orch,phsu', 'term': 'heparin', 'annotator': 'MetaMap'}
{'CUI': 'C0699992', 'semtype': 'orch,phsu', 'term': 'Lasix', 'annotator': 'MetaMap'}
{'CUI': 'C0004057', 'semtype': 'orch,phsu', 'term': 'aspirin', 'annotator': 'MetaMap'}
{'CUI': 'C0070166', 'semtype': 'orch,phsu', 'term': 'clopidogrel', 'annotator': 'MetaMap'}
{'CUI': 'C0042313', 'semtype': 'aapp,antb', 'term': 'vancomycin', 'annotator': 'MetaMap'}
{'CUI': 'C0633084', 'semtype': 'orch,phsu', 'term': 'Plavix', 'annotator': 'MetaMap'}
{'CUI': 'C0033487', 'semtype': 'orch,phsu', 'term': 'propofol', 'annotator': 'MetaMap'}
{'CUI': 'C0700776', 'semtype': 'orch,phsu', 'term': 'Lopressor', 'annotator': 'MetaMap', 'negation': 'True'}
Present,  value = True
{'CUI': 'C0065374', 'semtype': 'aapp,phsu', 'term': 'lisinopril', 'ann

{'CUI': 'C0025859', 'semtype': 'orch,phsu', 'term': 'metoprolol', 'annotator': 'MetaMap'}
{'CUI': 'C0070166', 'semtype': 'orch,phsu', 'term': 'clopidogrel', 'annotator': 'MetaMap'}
{'CUI': 'C0025859', 'semtype': 'orch,phsu', 'term': 'metoprolol', 'annotator': 'MetaMap'}
{'CUI': 'C0699129', 'semtype': 'orch,phsu', 'term': 'Coumadin', 'annotator': 'MetaMap'}
{'CUI': 'C0019134', 'semtype': 'bacs,orch,phsu', 'term': 'heparin', 'annotator': 'MetaMap'}
{'CUI': 'C0065374', 'semtype': 'aapp,phsu', 'term': 'lisinopril', 'annotator': 'MetaMap'}
{'CUI': 'C0699129', 'semtype': 'orch,phsu', 'term': 'Coumadin', 'annotator': 'MetaMap'}
{'CUI': 'C0699992', 'semtype': 'orch,phsu', 'term': 'Lasix', 'annotator': 'MetaMap'}
{'CUI': 'C0065374', 'semtype': 'aapp,phsu', 'term': 'lisinopril', 'annotator': 'MetaMap'}
{'CUI': 'C0065374', 'semtype': 'aapp,phsu', 'term': 'lisinopril', 'annotator': 'MetaMap'}
{'CUI': 'C0004057', 'semtype': 'orch,phsu', 'term': 'aspirin', 'annotator': 'MetaMap'}
{'CUI': 'C0065374',

In [18]:
negbio_pred.head()

,Lasix,Plavix,aspirin,captopril,clopidogrel,heparin,metoprolol,vancomycin,Lopressor,isosorbide,lisinopril,propofol,Antibiotics,Coumadin,Toprol-XL
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
3,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


### ¿y para extraer todos los medicamentos?

Usando la herramienta ligera de UMLS, conocida como: [MetaMapLite](https://metamap.nlm.nih.gov/MetaMapLite.shtml).
- MetaMapLite puede instarlarse localmente y permite el reconocimiento de entidades (medicamentos).

- Para usar MetaMapLite de forma local debe usarse la siguiente instrucción: 
`./metamaplite.sh --restrict_to_sts=orch,phsu notas/pat_100.txt`

- Para identificar diferentes tipos semánticos ver la [lista completa](https://metamap.nlm.nih.gov/Docs/SemanticTypes_2018AB.txt)